In [44]:
import yahooquery
import pandas as pd
from sklearn import dummy, metrics, model_selection, preprocessing
from sklearn import model_selection
#from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
#from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
#from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LinearRegression
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from pandas_datareader import data
import numpy as np

In [2]:
def get_yahoo_news(symbol):
    #start_date = date(2020, 6, 1)
    #end_date = date(2021, 1, 29)
    data = []
    run_Dict = []
    ticker = yahooquery.Ticker(symbol)
      
    news = ticker.news(2000,start="2021-01-29")
    news = pd.DataFrame(news)
    
    news=news[["provider_publish_time","title", "summary"]]
    #news['date'] = pd.fr(news['provider_publish_time']).dt.strftime('%m-%d-%Y')
    news = news.sort_values(by=['provider_publish_time'])
    news['date'] = pd.to_datetime(news['provider_publish_time'],unit='s').dt.strftime("%Y-%m-%d")
    news.set_index(pd.to_datetime(news['date']), inplace=True)
    news.drop('date', axis=1)
    nltk.download("vader_lexicon")
    sentimentAnalyser = SentimentIntensityAnalyzer()

    news["compound"] = [sentimentAnalyser.polarity_scores(v)['compound'] if isinstance(v,str) else v for v in news["summary"]]
    news["negative"] = [sentimentAnalyser.polarity_scores(v)['neg'] if isinstance(v,str) else v for v in news["summary"]]
    news["positive"] = [sentimentAnalyser.polarity_scores(v)['pos'] if isinstance(v,str) else v for v in news["summary"]]
    news["neutral"] = [sentimentAnalyser.polarity_scores(v)['neu'] if isinstance(v,str) else v for v in news["summary"]]


    #news['date'] = pd.to_datetime(news['date']).
    print(news.columns)
    return news

def get_sentiment_from_news(news):
    news_sentiment = news[['date', 'compound']]
    news_sentiment.set_index(pd.to_datetime(news_sentiment['date']), inplace=True)
    news_sentiment = news_sentiment['compound']
    total_sentiment = news_sentiment.groupby('date').agg(lambda x: sum(x))
    total_sentiment = total_sentiment.rename('Total Sentiment')
    nArticles = news_sentiment.groupby('date').count()
    avg_sentiment = total_sentiment/nArticles
    avg_sentiment = avg_sentiment.rename('Avg Sentiment')
    sentiment_data = pd.concat([total_sentiment, avg_sentiment], axis=1)
    sentiment_data = sentiment_data.reindex(df.index, method='ffill')
    return sentiment_data

In [3]:
def get_yahoo_dataframe(symbol):
  # Get historical data
  #symbol = 'AAPL'
    ticker = yahooquery.Ticker(symbol)
    df = pd.DataFrame(ticker.history(start='2020-01-29', end='2021-01-29'))
    df=data.DataReader(symbol, 'yahoo', '20200129', '20210129').reset_index()
    #df = df[['Date','Adj Close', 'Volume']]
    #(df[:10].style.format(dict(Date='{:%d/%m/%Y}'))
    #  .background_gradient(subset='Volume',cmap='Blues')
    #  .background_gradient(subset='Adj Close',cmap='Blues')
    #)
    #df.Volume.plot()
    #Volume = df['Volume']/10000000
    #Volume.plot.bar()
    #df["Adj Close"].plot()
    #dates = pd.DataFrame(pd.date_range('20200203',periods=8*31))
    df.set_index(pd.to_datetime(df['Date']), inplace=True)
    df = df.drop(['Date'], axis=1)
    return df
def process_stock_data(filename, df):
    stock_file = pd.read_csv(filename)
    stock_file.set_index(pd.to_datetime(stock_file['Date']), inplace=True)
    stock_file.drop('Date', axis=1)
    stock_file = stock_file.reindex(df.index, method='bfill')
    stock_file = stock_file.drop('Date', axis=1)
    stock_file = stock_file.drop('Period Ending', axis=1)
    return stock_file

In [4]:
def prepare_data_Y_for_ML(df):
    output = df['Adj Close']
    output.rename('Stock Price')
    advance = df['Adj Close'].shift(-10)
    advance = advance.rename('Short Term Advance')
    advance2 = df['Adj Close'].shift(-50)
    advance2 = advance2.rename('Mid Term Advance')
    gt = pd.concat([output, advance, advance2], axis=1)
    df['Short Term Change'] = ((gt['Short Term Advance'] - gt['Adj Close'])/gt['Adj Close'])*100
    df['Mid Term Change'] = ((gt['Mid Term Advance'] - gt['Adj Close'])/gt['Adj Close'])*100
    #short term 3% mid term 5% long term 10%
    df['stm'] = df['Short Term Change']
    df.loc[df['stm'] > 3, 'stm' ] = 3
    df.loc[df['stm'] < -3, 'stm' ] = -3
    df.loc[(df['stm']>-3)& (df['stm']<3),'stm'] = 0
    df.loc[df['stm'] == 3, 'stm' ] = 'up'
    df.loc[df['stm'] == -3, 'stm' ] = 'down'
    df.loc[df['stm']==0,'stm'] = 'neutral'
    #df[df['stm']<-3]=-3
    #df[(df['stm']>-3) & (df['stm']<3)]=0
      #print(gt[9:110])
      #gt['Short Term Change'].plot()

    #Y = gt[['Short Term Change']]
    #return Y
def prepare_data_X_for_ML(df):
    df['Weekly Sentiment'] = df['Avg Sentiment'].rolling(5, win_type='triang').sum()
    df['Monthly Sentiment'] = df['Avg Sentiment'].rolling(20, win_type='triang').sum()
    avg_monthly_sentiment = df['Monthly Sentiment'].sum()/len(df['Monthly Sentiment'])
    avg_week = df['Weekly Sentiment'].sum()/len(df['Monthly Sentiment'])
    #X = df[10:]
    df['Monthly Sentiment'] =  df['Monthly Sentiment'].fillna(avg_monthly_sentiment)
    df['Weekly Sentiment'] = df['Weekly Sentiment'].fillna(avg_week)
    #X=X.drop(['Date','Estimated_Revenue','Reported_Revenue','Estimated_EPS','Reported_EPS','Period Ending'],axis=1)
    #return X

In [5]:
#main code
symbol='AAPL'
df = get_yahoo_dataframe(symbol)
df
stock_file=process_stock_data('/Users/sagarjogadhenu/Downloads/TinyEarn/AAPL.csv', df)
#print(stock_file)
df = df.join(stock_file)
news = get_yahoo_news(symbol)
sentiment_data=get_sentiment_from_news(news)
#print(sentiment_data)
df = df.join(sentiment_data)
df[['Total Sentiment', 'Avg Sentiment']] = df[['Total Sentiment', 'Avg Sentiment']].fillna(method='ffill')
prepare_data_Y_for_ML(df)
prepare_data_X_for_ML(df)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/sagarjogadhenu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Index(['provider_publish_time', 'title', 'summary', 'date', 'compound',
       'negative', 'positive', 'neutral'],
      dtype='object')


In [6]:
#short term analysis
df = df.drop(columns='Mid Term Change')
df = df[df['stm'].notna()] # drop NA from stm
df['stm'] = df['stm'].astype('category')

In [7]:
df

,High,Low,Open,Close,Volume,Adj Close,Estimated_EPS,Reported_EPS,Surprise_EPS,Estimated_Revenue,Reported_Revenue,Total Sentiment,Avg Sentiment,Short Term Change,stm,Weekly Sentiment,Monthly Sentiment
Date,,,,,,,,,,,,,,,,,
2020-01-29,81.962502,80.345001,81.112503,81.084999,216229200.0,80.270790,1.14,1.25,0.11,88017.80,91819.0,NaN,NaN,1.121216,neutral,0.587579,1.870699
2020-01-30,81.022499,79.687500,80.135002,80.967499,126743200.0,80.154472,1.14,1.25,0.11,88017.80,91819.0,NaN,NaN,0.546820,neutral,0.587579,1.870699
2020-01-31,80.669998,77.072502,80.232498,77.377502,199588400.0,76.600517,1.14,1.25,0.11,88017.80,91819.0,NaN,NaN,5.237703,up,0.587579,1.870699
2020-02-03,78.372498,75.555000,76.074997,77.165001,173985600.0,76.390160,1.14,1.25,0.11,88017.80,91819.0,NaN,NaN,3.595236,up,0.587579,1.870699
2020-02-04,79.910004,78.407501,78.827499,79.712502,136616400.0,78.912079,1.14,1.25,0.11,88017.80,91819.0,NaN,NaN,1.736877,neutral,0.587579,1.870699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-08,132.630005,130.229996,132.429993,132.050003,105158200.0,131.852966,0.69,0.73,0.04,63388.91,64698.0,0.0000,0.000000,8.231738,up,1.940400,2.078669
2021-01-11,130.169998,128.500000,129.190002,128.979996,100620900.0,128.787552,0.69,0.73,0.04,63388.91,64698.0,0.7469,0.025755,10.993954,up,1.671785,2.176127
2021-01-12,129.690002,126.860001,128.500000,128.800003,91951100.0,128.607819,0.69,0.73,0.04,63388.91,64698.0,7.8081,0.260270,10.295034,up,0.935527,2.294586


In [8]:
df.columns

Index(['High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close', 'Estimated_EPS',
       'Reported_EPS', 'Surprise_EPS', 'Estimated_Revenue', 'Reported_Revenue',
       'Total Sentiment', 'Avg Sentiment', 'Short Term Change', 'stm',
       'Weekly Sentiment', 'Monthly Sentiment'],
      dtype='object')

In [73]:
# Pick columns of interest and split data for training and testing
x_cols = ['Volume','Estimated_EPS','Reported_EPS', 'Surprise_EPS', 'Estimated_Revenue', 'Reported_Revenue', 'Weekly Sentiment']
#x_cols = ['Volume', 'Weekly Sentiment']

y_cols = ['stm']
X_train, X_test, y_train, y_test = model_selection.train_test_split(df[x_cols],df[y_cols]\
                                                                                  , test_size=0.3)
cols = X_train.columns
std = preprocessing.StandardScaler()
X_train.loc[:, cols] = std.fit_transform(X_train)
X_test.loc[:,cols] = std.transform(X_test)

In [74]:

lr= LogisticRegression(multi_class='multinomial',solver='sag')
lr.fit(X_train,np.ravel(y_train))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

In [75]:
lr.score(X_test,np.ravel(y_test))

0.5540540540540541

In [76]:
from sklearn import model_selection
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [77]:
dt = DecisionTreeClassifier()

In [78]:
dt.fit(X_train,np.ravel(y_train))
dt.score(X_test,np.ravel(y_test))

0.4189189189189189

In [79]:
kn = KNeighborsClassifier()

In [80]:
kn.fit(X_train,np.ravel(y_train))
kn.score(X_test,np.ravel(y_test))

0.5135135135135135

In [81]:
rf = RandomForestClassifier()
rf.fit(X_train,np.ravel(y_train))
rf.score(X_test,np.ravel(y_test))

/Users/sagarjogadhenu/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.5135135135135135

In [82]:
gn = GaussianNB()
gn.fit(X_train,np.ravel(y_train))
gn.score(X_test,np.ravel(y_test))

0.5675675675675675

In [83]:
sv = SVC()
sv.fit(X_train,np.ravel(y_train))
sv.score(X_test,np.ravel(y_test))

0.5945945945945946